<a href="https://colab.research.google.com/github/shivanishingne/Driver-drowsiness-Detection/blob/master/detect_drowsiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Implementing Driver Drowsiness Detection algorithm using OpenCV, dlib, and Python:*

---



1.   Importing the necessary packages:



In [ ]:
!pip install --upgrade playsound
from scipy.spatial import distance as dist
import imutils
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
from playsound import playsound
import dlib
import numpy as np
import argparse
import time
import cv2
import argparse

*   **SciPy**: We’ll need the SciPy package so we can compute the Euclidean distance between facial landmarks points in the Eye Aspect Ratio calculation.
*   **imutils**: We’ll also use the imutils package, a series of computer vision and image processing functions to make working with OpenCV easier.
* **Thread**: We'll import the Thread class so we can play our alarm in a separate thread from the main thread to ensure our script doesn’t pause execution while the alarm sounds.
* **playsound**: We'll need the playsound library to play simple sounds like our MP3 alarm.
* **dlib**: To detect and localize facial landmarks we’ll need the dlib library.




---
2. Defining the sound_alarm function, which accepts a path to an audio file on the disk and then play  the sound:

In [ ]:
def sound_alarm(path):
  # play an alarm sound
  playsound.playsound(path)

---
3. Defining the `eye_aspect_ratio` function which is used to compute the ratio of distances between the vertical eye landmarks and the distances between the horizontal eye landmarks:

In [ ]:
def eye_aspect_ratio(eye):
  # compute the Euclidean distances between the two sets of vertical eye lankmark (x,y) coordinates:
  A = dist.euclidean(eye[1], eye[5])
  B = dist.euclidean(eye[2], eye[4])

  # compute the Euclidean distances between the two sets of horizontal eye lankmark (x,y) coordinates:
  C = dist.euclidean(eye[0], eye[3])

  # compute the Eye-Aspect-Ratio:
  ear = (A+B) / (2.0 * C)

  # return the E.A.R:
  return ear

* The EAR will be approximately constant when the eye is open, and when the eye is closed. However, the ratio will be much smaller than the ratio when the eye is open.
* During a blink, the value will rapidly decrease towawrds zero.
(Ref: Soukupová and Čech’s 2016 paper, [Real-Time Eye Blink Detection using Facial Landmarks](http://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf))

* In our drowsiness detector case, we’ll be monitoring the eye aspect ratio to see if the value *falls* but does *not increase again*, thus implying that the person has closed their eyes.


---
4. constructing the argument parser and parsing the arguments:

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
ap.add_argument("-a", "--alarm", type=str, default="", help="path alarm .WAV file")
ap.add_argument("-w", "--webcam", type=int, default=0, help="index of webcam on system")
args = vars(ap.parse_args())

* `--shape-predictor` : This is the path to dlib’s pre-trained facial landmark detector.
* `--alarm` : Here you can optionally specify the path to an input audio file to be used as an alarm.
* `--webcam` : This integer controls the index of your built-in webcam/USB camera.

---
5. Defining constants:

In [ ]:
# initialize the threshold for the EAR, indicating a "blink"
EYE_AR_THRESH = 0.3

# initialize the threshold for the number of consecutive frames to set off the alarm
EYE_AR_CONSEC_FRAMES = 48

# initialize the frame counter
COUNTER = 0

# initialize a boolean used to indicate if the alarm is going off
ALARM_ON = False

- `EYE_AR_THRESH`: If the eye aspect ratio falls below this threshold, we’ll start counting the number of frames the person has closed their eyes for.
- `EYE_AR_CONSEC_FRAMES`: If the number of frames the person has closed their eyes in exceeds EYE_AR_CONSEC_FRAMES, we’ll sound an alarm.
- `COUNTER`: COUNTER defines the total number of consecutive frames where the eye aspect ratio is below EYE_AR_THRESH.
- `ALARM_ON`: We'll update the boolean ALARM_ON if COUNTER exceeds EYE_AR_CONSEC_FRAMES.

---

6. Instantiate dlib's Histogram of Oriented Gradient-based face detector, along with Facial Landmark Predictor:

In [ ]:
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])